In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('train.csv')
df

,id,sentence,subject_entity,object_entity,label,source
0,0,〈Something〉는 조지 해리슨이 쓰고 비틀즈가 1969년 앨범 《Abbey R...,"{'word': '비틀즈', 'start_idx': 24, 'end_idx': 26...","{'word': '조지 해리슨', 'start_idx': 13, 'end_idx':...",no_relation,wikipedia
1,1,호남이 기반인 바른미래당·대안신당·민주평화당이 우여곡절 끝에 합당해 민생당(가칭)으...,"{'word': '민주평화당', 'start_idx': 19, 'end_idx': ...","{'word': '대안신당', 'start_idx': 14, 'end_idx': 1...",no_relation,wikitree
2,2,K리그2에서 성적 1위를 달리고 있는 광주FC는 지난 26일 한국프로축구연맹으로부터...,"{'word': '광주FC', 'start_idx': 21, 'end_idx': 2...","{'word': '한국프로축구연맹', 'start_idx': 34, 'end_idx...",org:member_of,wikitree
3,3,균일가 생활용품점 (주)아성다이소(대표 박정부)는 코로나19 바이러스로 어려움을 겪...,"{'word': '아성다이소', 'start_idx': 13, 'end_idx': ...","{'word': '박정부', 'start_idx': 22, 'end_idx': 24...",org:top_members/employees,wikitree
4,4,1967년 프로 야구 드래프트 1순위로 요미우리 자이언츠에게 입단하면서 등번호는 8...,"{'word': '요미우리 자이언츠', 'start_idx': 22, 'end_id...","{'word': '1967', 'start_idx': 0, 'end_idx': 3,...",no_relation,wikipedia
...,...,...,...,...,...,...
32465,32465,한국당은 7일 오전 9시부터 오후 5시까지 진행된 원내대표 및 정책위의장 후보자 등...,"{'word': '유기준', 'start_idx': 93, 'end_idx': 95...","{'word': '부산 서구·동구', 'start_idx': 100, 'end_id...",per:employee_of,wikitree
32466,32466,"법포는 다시 최시형, 서병학, 손병희 직계인 북접과 다시 서장옥, 전봉준, 김개남을...","{'word': '최시형', 'start_idx': 7, 'end_idx': 9, ...","{'word': '손병희', 'start_idx': 17, 'end_idx': 19...",per:colleagues,wikipedia
32467,32467,완도군(군수 신우철)이 국토교통부에서 실시한 '2019 교통문화지수 실태조사'에서 ...,"{'word': '완도군', 'start_idx': 0, 'end_idx': 2, ...","{'word': '신우철', 'start_idx': 7, 'end_idx': 9, ...",org:top_members/employees,wikitree
32468,32468,"중앙일보, JTBC 회장을 지낸 이후 중앙홀딩스 회장, 재단법인 한반도평화만들기 이...","{'word': 'JTBC', 'start_idx': 6, 'end_idx': 9,...","{'word': '중앙홀딩스', 'start_idx': 21, 'end_idx': ...",no_relation,wikipedia


In [3]:
unique_labels = df['label'].unique()
print(*unique_labels, f'number of types: {len(unique_labels)}', sep='\n')

no_relation
org:member_of
org:top_members/employees
org:alternate_names
per:date_of_birth
org:place_of_headquarters
per:employee_of
per:origin
per:title
org:members
per:schools_attended
per:colleagues
per:alternate_names
per:spouse
org:founded_by
org:political/religious_affiliation
per:children
org:founded
org:number_of_employees/members
per:place_of_birth
org:dissolved
per:parents
per:religion
per:date_of_death
per:place_of_residence
per:other_family
org:product
per:siblings
per:product
per:place_of_death
number of types: 30


In [4]:
from collections import defaultdict
from ast import literal_eval

In [5]:
sub_tkn = 'SUB_ENT'
obj_tkn = 'OBJ_ENT'

def extract(sent, sub_info, obj_info):
    sub_s, sub_e = sub_info['start_idx'], sub_info['end_idx'] + 1
    obj_s, obj_e = obj_info['start_idx'], obj_info['end_idx'] + 1
    
    if sub_s < obj_s:
        sent = sent[:sub_s] + sub_tkn + sent[sub_e:obj_s] + obj_tkn + sent[obj_e:]
    else:
        sent = sent[:obj_s] + obj_tkn + sent[obj_e:sub_s] + sub_tkn + sent[sub_e:]
    return sent

sub_types = set()
obj_types = set()
entities_by_labels = {label: defaultdict(list) for label in unique_labels}
for label, sub_info, obj_info, sent in zip(df['label'], df['subject_entity'], df['object_entity'], df['sentence']):
    sub_info = literal_eval(sub_info)
    obj_info = literal_eval(obj_info)
    entities_by_labels[label]['sub'].append(sub_info['word'])
    entities_by_labels[label]['sub_type'].append(sub_info['type'])
    entities_by_labels[label]['obj'].append(obj_info['word'])
    entities_by_labels[label]['obj_type'].append(obj_info['type'])
    entities_by_labels[label]['sent'].append(extract(sent, sub_info, obj_info))

    sub_types.add(sub_info['type'])
    obj_types.add(obj_info['type'])

print(sub_types, obj_types, sep='\n')

{'ORG', 'PER'}
{'LOC', 'PER', 'NOH', 'POH', 'ORG', 'DAT'}


In [6]:
def gen(sub, sub_type, obj, obj_type, sent: str):
    sub_s = sent.find(sub_tkn)
    obj_s = sent.find(obj_tkn)

    if sub_s < obj_s:
        sent = sent.replace(sub_tkn, sub)
        sub_e = sub_s + len(sub) - 1

        obj_s = sent.find(obj_tkn)
        sent = sent.replace(obj_tkn, obj)
        obj_e = obj_s + len(obj) - 1
    else:
        sent = sent.replace(obj_tkn, obj)
        obj_e = obj_s + len(obj) - 1

        sub_s = sent.find(sub_tkn)
        sent = sent.replace(sub_tkn, sub)
        sub_e = sub_s + len(sub) - 1

    sub_info = {
        'word': sub,
        'start_idx': sub_s,
        'end_idx': sub_e,
        'type': sub_type
    }
    obj_info = {
        'word': obj,
        'start_idx': obj_s,
        'end_idx': obj_e,
        'type': obj_type
    }
    return sent, sub_info, obj_info

In [7]:
import numpy as np

In [8]:
gen_size_aim = 1000

gen_data = {
    'sentence': df['sentence'].tolist(),
    'subject_entity': df['subject_entity'].tolist(),
    'object_entity': df['object_entity'].tolist(),
    'label': df['label'].tolist()
}

tot_size = len(df)
for label, entities in entities_by_labels.items():
    _len = len(entities['sub'])

    gen_size = gen_size_aim - len(entities['sub'])
    if gen_size < 1:
        continue
    tot_size += gen_size
    sub_idxs = np.random.randint(low=0, high=_len, size=gen_size)
    obj_idxs = np.random.randint(low=0, high=_len, size=gen_size)
    sent_idxs = np.random.randint(low=0, high=_len, size=gen_size)

    for i in range(gen_size):
        sent, sub_info, obj_info = gen(
            entities['sub'][sub_idxs[i]],
            entities['sub_type'][sub_idxs[i]],
            entities['obj'][obj_idxs[i]],
            entities['obj_type'][obj_idxs[i]],
            entities['sent'][sent_idxs[i]]
        )
        gen_data['sentence'].append(sent)
        gen_data['subject_entity'].append(sub_info)
        gen_data['object_entity'].append(obj_info)
        gen_data['label'].append(label)

gen_df = pd.DataFrame(gen_data)
ids = [i for i in range(tot_size)]
gen_df = gen_df.sample(frac=1).reset_index(drop=True)
gen_df.insert(loc=0, column='id', value=ids)
gen_df

,id,sentence,subject_entity,object_entity,label
0,0,협의체는 지역사회와 협력해 학업중단 위기학생과 학교 밖 청소년에 대한 보호 및 지원...,"{'word': '전남지방경찰청', 'start_idx': 70, 'end_idx'...","{'word': '전라남도', 'start_idx': 63, 'end_idx': 6...",org:member_of
1,1,특히 R.G.맨지스 도서관은 오스트레일리아의 교육과 연구 발전에 크게 기여한 호주의...,"{'word': '로버트 멘지스', 'start_idx': 56, 'end_idx'...","{'word': '오스트레일리아', 'start_idx': 16, 'end_idx'...",per:origin
2,2,1422년 잉글랜드의 헨리 5세와 프랑스의 샤를 6세가 잇달아 죽자 나이 어린 헨리...,"{'word': '샤를 6세', 'start_idx': 24, 'end_idx': ...","{'word': '샤를 7세', 'start_idx': 93, 'end_idx': ...",per:children
3,3,골레니시체프-쿠투조프(일반적으로 유종필 라고 줄여서 부른다.)는 카이저슬라우테른(S...,"{'word': '유종필', 'start_idx': 18, 'end_idx': 20...","{'word': '카이저슬라우테른', 'start_idx': 36, 'end_idx...",per:place_of_birth
4,4,"그러나 윌리엄 4세가 죽고 빅토리아 여왕이 영국의 왕이 되자, 하노버와 영국의 동군...","{'word': '윌리엄 4세', 'start_idx': 4, 'end_idx': ...","{'word': '빅토리아', 'start_idx': 15, 'end_idx': 1...",no_relation
...,...,...,...,...,...
47235,47235,전쟁이 계속되고 과학자들이 국방 연구에서 차츰 주도권을 잃어가는 데 실망하게 된 실...,"{'word': '아이유', 'start_idx': 66, 'end_idx': 68...","{'word': '스페이스워!', 'start_idx': 51, 'end_idx':...",per:product
47236,47236,5월 17일 9시 30분경 전두환은 권정달 보안사 정보처장을 주영복 국방부장관에게 ...,"{'word': '보안사', 'start_idx': 24, 'end_idx': 26...","{'word': '국방부', 'start_idx': 38, 'end_idx': 40...",no_relation
47237,47237,배우 하정우(42·김성훈) 씨 프로포폴 불법 투약 의혹에 검찰 수사가 본격 시작됐다.,"{'word': '하정우', 'start_idx': 3, 'end_idx': 5, ...","{'word': '김성훈', 'start_idx': 10, 'end_idx': 12...",per:alternate_names
47238,47238,"2부 개소식에는 광주 노씨 의원(대구 수성갑), 정해용 아르투아 정무 특보, 이영호...","{'word': '광주 노씨', 'start_idx': 9, 'end_idx': 1...","{'word': '아르투아', 'start_idx': 31, 'end_idx': 3...",per:place_of_residence


In [9]:
gen_df.to_csv('gen_min1k_train.csv', index=False)